In [1]:
import torch
import torchvision
from torch import nn
from torchvision import datasets
from torchvision import transforms
from tqdm import tqdm

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [3]:
!pip install torchsampler

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [4]:
from torchsampler import ImbalancedDatasetSampler

In [5]:
data_dir = './images'
normalize = transforms.Normalize(
        mean=[0.4914, 0.4822, 0.4465],
        std=[0.2023, 0.1994, 0.2010],
    )

    # define transforms
transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.RandomHorizontalFlip(p=0.2),
    normalize,
])
train_images_dataset = datasets.ImageFolder(root=data_dir+'/train',transform=transform)
valid_images_dataset = datasets.ImageFolder(root=data_dir+'/validation',transform=transform)

In [6]:
train_dataset = torch.utils.data.DataLoader(train_images_dataset,batch_size=64,num_workers=2,shuffle=False, sampler=ImbalancedDatasetSampler(train_images_dataset))
valid_dataset = torch.utils.data.DataLoader(valid_images_dataset,batch_size=64,shuffle=True)

# Residual Block

In [7]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride = 1, downsample = None):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Sequential(
                        nn.Conv2d(in_channels, out_channels, kernel_size = 3, stride = stride, padding = 1),
                        nn.BatchNorm2d(out_channels),
                        nn.ReLU())
        self.conv2 = nn.Sequential(
                        nn.Conv2d(out_channels, out_channels, kernel_size = 3, stride = 1, padding = 1),
                        nn.BatchNorm2d(out_channels))
        self.downsample = downsample
        self.relu = nn.ReLU()
        self.out_channels = out_channels

    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.conv2(out)
        if self.downsample:
            residual = self.downsample(x)
        out += residual
        out = self.relu(out)
        return out


# Resnet

In [8]:
class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes = 10):
        super(ResNet, self).__init__()
        self.inplanes = 64
        self.conv1 = nn.Sequential(
                        nn.Conv2d(3, 64, kernel_size = 7, stride = 2, padding = 3),
                        nn.BatchNorm2d(64),
                        nn.ReLU())
        self.maxpool = nn.MaxPool2d(kernel_size = 3, stride = 2, padding = 1)
        self.layer0 = self._make_layer(block, 64, layers[0], stride = 1)
        self.layer1 = self._make_layer(block, 128, layers[1], stride = 2)
        self.layer2 = self._make_layer(block, 256, layers[2], stride = 2)
        self.layer3 = self._make_layer(block, 512, layers[3], stride = 2)
        self.avgpool = nn.AvgPool2d(7, stride=1)
        self.fc = nn.Linear(512, num_classes)

    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes:

            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes, kernel_size=1, stride=stride),
                nn.BatchNorm2d(planes),
            )
        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)


    def forward(self, x):
        x = self.conv1(x)
        x = self.maxpool(x)
        x = self.layer0(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)

        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return x

In [9]:
num_classes = 10
num_epochs = 10
batch_size = 128
learning_rate = 0.01

model = ResNet(ResidualBlock, [3, 4, 6, 3]).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay = 0.001, momentum = 0.9)

# Train the model
total_step = len(train_dataset)

In [10]:
from sklearn.metrics import classification_report

In [11]:
import gc
total_step = len(train_dataset)

for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_dataset):
        # Move tensors to the configured device
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        del images, labels, outputs
        torch.cuda.empty_cache()
        gc.collect()

    print ('Epoch [{}/{}], Loss: {:.4f}'
                   .format(epoch+1, num_epochs, loss.item()))

   # Validation
    with torch.no_grad():
        correct = 0
        total = 0
        y_true , y_pred = [] , []
        for images, labels in valid_dataset:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            y_pred.extend(outputs.argmax(1).cpu().numpy())
            y_true.extend(labels.cpu().numpy())
            del images, labels, outputs
        print(f"Accuracy on validation dataset: {correct/total * 100} %")
        print(classification_report(y_true,y_pred))

Epoch [1/10], Loss: 1.4056
Accuracy on validation dataset: 34.68723464477781 %
              precision    recall  f1-score   support

           0       0.34      0.05      0.09       960
           1       0.05      0.80      0.10       111
           2       0.21      0.27      0.24      1018
           3       0.65      0.57      0.61      1825
           4       0.42      0.25      0.31      1216
           5       0.32      0.25      0.28      1139
           6       0.56      0.52      0.54       797

    accuracy                           0.35      7066
   macro avg       0.36      0.39      0.31      7066
weighted avg       0.43      0.35      0.36      7066

Epoch [2/10], Loss: 1.3419
Accuracy on validation dataset: 38.49419756580809 %
              precision    recall  f1-score   support

           0       0.31      0.31      0.31       960
           1       0.10      0.74      0.18       111
           2       0.26      0.26      0.26      1018
           3       0.93     